### HKU STEM workshop 2022 - "Data Science Lab": how sequencing is used to identify SARS-CoV2 strains

*by Dr Jason Wong and Dr Joshua Ho*

The objective of this Data Science Lab is show demonstrate how sequencing data of viruses are used to identify coronavirus strains and to study their effect on the virus.


## Set working directory

By default working directoy will be My Drive/SARS-CoV2

In [ ]:
# set working pathway to your own google drive doc
from google.colab import drive
drive.mount('/content/gdrive')                         # if using for the first time, you be requested to grant permission to link your Google Drive

import os
try:
  os.mkdir("/content/gdrive/My Drive/SARS-CoV2")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/SARS-CoV2")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
directory already exist. OK to continue


In [ ]:
pwd

'/content/gdrive/MyDrive/SARS-CoV2'

## Package installation and downloads for workshop (~ 5 minutes)

1.   conda (for simple installation of packages)
2.   ClustalW (for sequence alignment)
3.   biopython (tools for interacting with bioinformatics file formats)
4.   py3Dmol (for protein structure visualisation)
5.   Download ready prepared files for analysis.


In [ ]:
# install conda (~ 1 min)
!pip install -q condacolab
import condacolab#
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:34
🔁 Restarting kernel...


In [ ]:
# install ClustalW
!conda install -c bioconda clustalw

Solving environment: - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - clustalw


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       ha878542_0         149 KB  conda-forge
    certifi-2022.6.15          |   py37h89c1867_0         155 KB  conda-forge
    clustalw-2.1               |       h7d875b9_6         318 KB  bioconda
    conda-4.13.0               |   py37h89c1867_1         989 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.6 MB

The following NEW packages will be INSTALLED:

  clustalw           bioconda/linux-64::clustalw-2.1-h7d875b9_6

The following packages will be UPDATED:

  ca-certificates                      2020.12.5-ha878542_0 --> 2022.6.15-ha878542_0
  certifi                

In [ ]:
# install Biopython
# note that there maybe a message about the need for a restart due to a new version of numpy
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached biopython-1.79-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (2.3 MB)
     |████████████████████████████████| 15.7 MB 14.9 MB/s 


In [ ]:
# install py3Dmol
!pip -q install py3Dmol


In [ ]:
# double check that we are in right directory
os.chdir("/content/gdrive/My Drive/SARS-CoV2")

# download fasta files for analysis
!wget -O SARSCoV2_Spike.fa https://cloudstor.aarnet.edu.au/plus/s/qXWe5qWpaA4PhUC/download        # SARSCov2_Spike.fa
!wget -O Virus_seq.fa https://cloudstor.aarnet.edu.au/plus/s/VQYNF5YDNa5g2fZ/download        # Virus_seq.fa

--2022-07-09 08:33:28--  https://cloudstor.aarnet.edu.au/plus/s/qXWe5qWpaA4PhUC/download
Resolving cloudstor.aarnet.edu.au (cloudstor.aarnet.edu.au)... 202.158.207.20
Connecting to cloudstor.aarnet.edu.au (cloudstor.aarnet.edu.au)|202.158.207.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: 5230042dc1897=uue751ljgvs5nni9v1t0ku0dae; path=/plus;; Secure at position 53.
Syntax error in Set-Cookie: oc_sessionPassphrase=DT%2Fnl8T3eCURDFnY7a%2FrI1JhQaI06CY3LSq03vWuGUWgV4I%2BT8Q9Wtd19CXy%2FL5nFiB4nwYTM0Iw%2F%2FERLzYmo%2BG30jsvOqMNNbENcDDxONzMgeJkpsQObmrPEgZ3ugR%2B; expires=Sun, 10-Jul-2022 08:33:30 GMT; Max-Age=86400; path=/plus;; Secure at position 232.
Length: 8333 (8.1K) [application/octet-stream]
Saving to: ‘SARSCoV2_Spike.fa’

SARSCoV2_Spike.fa   100%[===================>]   8.14K  --.-KB/s    in 0.02s   

2022-07-09 08:33:30 (487 KB/s) - ‘SARSCoV2_Spike.fa’ saved [8333/8333]

--2022-07-09 08:33:30--  https://cloudstor.aarnet.edu.au/plus/s/

In [ ]:
ls

condacolab_install.log  SARSCov2_Spike.fa  Virus_seq.fa


## Whole genome alignment among virus strains

We have prepared sequences for 6 well characterised coronavirus species. These are:

*   SARS-CoV2
*   SARS-CoV
*   AlphaCoV 229E
*   AlphaCoV NL63
*   MERS

For this first section, we will demonstrate how the genome sequence of the viruses can be used to distingush and reveal their relationship.

1.   Run multiple sequence alignment on the virus sequences
2.   Count the number of differing nucleotides between SARS-CoV2 and the other virues
3.   Construct a phylogenetic tree



In [ ]:
# take a look at Fasta file format
!head Virus_seq.fa

>SARSCoV2_NC045512
...
>AlphaCoV_229E
...
>AlphaCoV_NL63
...
>MERS_NC019843.3
...
>SARSCoV_NC004718
...


In [ ]:
# run ClustalW for alignment. (~3 minutes)
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
cmd = ClustalwCommandline("clustalw2",infile="Virus_seq.fa",topdiags=1,quicktree=1)
print(cmd)
stdout, stderr = cmd()

# check to see output files generated by ClustalW. Should see two new files Virus_seq.aln and Virus_seq.dnd
!ls

clustalw2 -infile=Virus_seq.fa -quicktree -topdiags=1
condacolab_install.log	SARSCoV2_Spike.dnd  SARSCoV2_Spike.fa  Virus_seq.dnd
SARSCoV2_Spike.aln	SARSCov2_Spike.fa   Virus_seq.aln      Virus_seq.fa


In [ ]:
# Display alignment result as text output. First 50 lines.
align = open("Virus_seq.aln", "r")
print(''.join(align.readlines()[0:100]))

CLUSTAL 2.1 multiple sequence alignment


SARSCoV2_NC045512      ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT
SARSCoV_NC004718       ATATTAGGTTTTTACCTACCCAGGAAA--AGCCAACCAACCT-CGATCTCTTGTAGATCT
MERS_NC019843.3        -------GATTTAAGTGAATAGCTTGGCTATCTCACTTCCCCTCGTTCTCTTGCAGAACT
AlphaCoV_229E          ------------------------------------------------------------
AlphaCoV_NL63          ------------------------------------------------------------
                                                                                   

SARSCoV2_NC045512      GTTCTCTAAACGAACTT---TAAAATCTGTGTGGCT-GTCACTCGGCTGCA--TGCTTAG
SARSCoV_NC004718       GTTCTCTAAACGAACTT---TAAAATCTGTGTAGCT-GTCGCTCGGCTGCA--TGCCTAG
MERS_NC019843.3        TTGATTTTAACGAACTTAAATAAAAGCCCTGTTGTTTAGCGTATCGTTGCACTTGTCTGG
AlphaCoV_229E          ------------------------------------------------------------
AlphaCoV_NL63          ------------------------------------------------------------
                                 

In [ ]:
# Count number of nucleotides that are different between SARS-CoV2 and each virus
# read alignment result
from Bio import AlignIO
align = AlignIO.read("Virus_seq.aln", "clustal")
print(align)

# Get SARSCoV2 record
SARSCoV2 = align[0]

# Loop through and count differences
for i in align:
  diff_nuc = 0
  pos = 0
  for j in i.seq:
    if (j != "-" or SARSCoV2.seq[pos] != "-") and j != SARSCoV2.seq[pos]:
       diff_nuc+=1
    pos+=1
  print(i.id, diff_nuc)  

Alignment with 5 rows and 30963 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...AAA SARSCoV2_NC045512
ATATTAGGTTTTTACCTACCCAGGAAA--AGCCAACCAACCT-C...--- SARSCoV_NC004718
-------GATTTAAGTGAATAGCTTGGCTATCTCACTTCCCCTC...--- MERS_NC019843.3
--------------------------------------------...--- AlphaCoV_229E
--------------------------------------------...--- AlphaCoV_NL63
SARSCoV2_NC045512 0
SARSCoV_NC004718 6248
MERS_NC019843.3 14987
AlphaCoV_229E 17628
AlphaCoV_NL63 17376


In [ ]:
#Draw phylogenetic tree

from Bio import Phylo
tree = Phylo.read("Virus_seq.dnd", "newick")
Phylo.draw_ascii(tree)

                              _____________________ SARSCoV2_NC045512
  ___________________________|
 |                           |________________________ SARSCoV_NC004718
 |
_|                __________________________________________ AlphaCoV_229E
 |_______________|
 |               |_______________________________________ AlphaCoV_NL63
 |
 |_________________________________________________ MERS_NC019843.3



## Spike protein sequence analysis

The spike protein is of considerable interest in terms of SARS-CoV2 because it is responsible for recognising human cells and mediating entry and infection. Furthermore, COVID-19 vaccines are designed to generate antibodies that target the spike protein. Therefore mutations in the spike protein can both affect the virulance of SARS-CoV2 and may also affect the efficacy of vaccines.

For this part of the workshop we will specifically analyse the spike protein of SARS-CoV2 strains. Specifically we will:

1.   Translate the S gene into the spike protein sequence.
2.   Align and compare the spike protein for SARS-CoV2 strains.
3.   Find the specific amino acid differences between Wuhan strain with Delta and Omicron BA.5
4.   Visualise the spike protein structure.



In [ ]:
# Extract S gene and translate to protein sequence
# Based on NCBI the location of the Spike gene is between 21563..25384
from Bio import SeqIO
rawSeq = SeqIO.parse("Virus_seq.fa", "fasta")
for i in rawSeq:
  if i.id == "SARSCoV2_NC045512":
    print(i)
    Sgene = i.seq[21562:25384]              # python index are 0-based so need to subtract start by 1. End is exclusive so leave as is.

print("\nSARSCoV2_NC045512 S gene sequence")
print(Sgene)

#translate from DNA to Protein
Sprotein = Sgene.translate()
print("\nSARSCoV2_NC045512 spike protein sequence")
print(Sprotein)


ID: SARSCoV2_NC045512
Name: SARSCoV2_NC045512
Description: SARSCoV2_NC045512
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')

SARSCoV2_NC045512 S gene sequence
ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTTACAACCAGAACTCAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGCTATACATGTCTCTGGGACCAATGGTACTAAGAGGTTTGATAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGGTGTTTATTACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGGGTTTTTCGGCTTTAGAACCATTGGTAGATTTGCCAATAGGTATTAACATCACTAGGTTTCAAACTTTACTTGCTTTACATAGAAGTTATTTGACTCCTGGTGATTCTTCTTCAGGTTGGACAGCTGGTGCTGCAGCTTATTAT

In [ ]:
# run Clustalw and again aligned amino acid sequences with the same length
from Bio.Align.Applications import ClustalwCommandline
cmd = ClustalwCommandline("clustalw2",infile="SARSCoV2_Spike.fa")
print(cmd)
stdout, stderr = cmd()

# look at phylogenetic relationship
from Bio import Phylo
tree = Phylo.read("SARSCoV2_Spike.dnd", "newick")
Phylo.draw_ascii(tree)

# Display alignment result as text output (show only first 98 lines of the file).
align = open("SARSCoV2_Spike.aln", "r")
print(''.join(align.readlines()[0:98]))

clustalw2 -infile=SARSCoV2_Spike.fa
  , Wuhan.1
 ,|
 ||____________ Beta
 |
 | __________ Alpha
_||
 ||                                  _____________________________ Omicron.BA1
 ||_________________________________|
 |                                  |_________________ Omicron.BA5
 |
 |_____________ Delta

CLUSTAL 2.1 multiple sequence alignment


Wuhan.1          MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
Beta             MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
Delta            MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
Alpha            MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
Omicron.BA1      MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
Omicron.BA5      MFVFLVLLPLVSSQCVNLITRTQS---YTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
                 ****************** ****    *********************************

Wuhan.1          NVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIV
Beta             NVTWFH

In [ ]:
#Get differences in Spike protein between Wuhan and Delta
#read alignment result
from Bio import AlignIO
align = AlignIO.read("SARSCoV2_Spike.aln", "clustal")

print ("Delta variants in Spike protein")

#Get Wuhan record
Wuhan = align[0]

#Loop through and find mutations in Delta Spike protein
for i in align:
  if i.id == "Delta":
    pos = 0
    realpos = 0             # accounts for gaps in the Wuhan protein sequence
    for j in i.seq:
      if j != Wuhan.seq[pos]:
        print(Wuhan.seq[pos],str(realpos+1),j)
      if (Wuhan.seq[pos] != "-"):
          realpos+=1
      pos+=1 


print ("\nOmicron BA.5 variants in Spike protein")
#Loop through and find mutations in Omicron BA.5 Spike protein
for i in align:
  if i.id == "Omicron.BA5":
    pos = 0
    realpos = 0             # accounts for gaps in the Wuhan protein sequence
    for j in i.seq:
      if j != Wuhan.seq[pos]:
        print(Wuhan.seq[pos],str(realpos+1),j)
      if (Wuhan.seq[pos] != "-"):
          realpos+=1
      pos+=1 


Delta variants in Spike protein
T 19 R
K 77 T
E 156 G
F 157 -
R 158 -
L 452 R
T 478 K
D 614 G
P 681 R
D 950 N

Omicron BA.5 variants in Spike protein
T 19 I
L 24 S
P 25 -
P 26 -
A 27 -
H 69 -
V 70 -
T 76 I
G 142 D
V 213 G
R 214 -
- 215 R
G 339 D
S 371 F
S 373 P
S 375 F
T 376 A
D 405 N
R 408 S
K 417 N
N 440 K
L 452 R
S 477 N
T 478 K
E 484 A
F 486 V
Q 498 R
N 501 Y
Y 505 H
D 614 G
H 655 Y
N 679 K
P 681 H
N 764 K
D 796 Y
Q 954 H
N 969 K


In [ ]:
# Visualising the structure of the spike protein
# load cyroEM structure of spike protein from PBD (protein databank)

import py3Dmol
p = py3Dmol.view(query='mmtf:7BBH',width=600,height=600)
p.setBackgroundColor('grey')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.addStyle({'chain':'A'},{'cartoon': {'color':'red'}})
p.addStyle({'chain':'B'},{'cartoon': {'color':'green'}})
p.addStyle({'chain':'C'},{'cartoon': {'color':'blue'}})

p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#view RBD (receptor bind domain variants)
p = py3Dmol.view(query='mmtf:7BBH',width=600,height=600)
p.setBackgroundColor('grey')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.addStyle({'chain':'C','resi':["319-541"]},{'cartoon': {'color':'red'}})
p.addStyle({'chain':'C','resi':[452,478]},{'stick':{'colorscheme':'skyBlueCarbon'}})
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol